# Reconocimiento Facial

Código en Google Colab: https://colab.research.google.com/drive/1Ts_suIiKu7RqTKEgGUgo0XGb1gG_7Mnt?usp=sharing

Repositorio en Github:

## Ejemplo 1

En este ejemplo, vamos a realizar un reconocimiento facial pero utilizando imágenes cargadas desde nuestra computadora, y no en tiempo real. Utilizaremos un dataset con imágenes de 3 famosos panameños: Ruben Blades, Román Torres y Erika Ender. Para validar que el modelo funciona correctamente, utilizaremos una imagen de cada uno que no se encuentre en el Dataset

### Importación de Librerías

In [ ]:
#importación de la librería OpenCV
import cv2
#importación de la librería os para leer carpetas y archivos
import os
#importar numpy para convertir listas de python en matrices numpy
#Esto es necesario para el reconocimiento facial en Python
import numpy as np

### Asignación de etiquetas a cada clase

In [ ]:
#etiquetado de cada sujeto a los que evaluaremos con el reconocimiento facial. El primer espacio de este vector lo dejamos
#siempre vacio
subjects = ["", "Ruben Blades", "Erika Ender", "Román Torres"]

Cada sujeto que vamos a reconocer equivaldría a una clase, viendolo desde el punto de vista de un modelo de Machine Learning.

### Detección del rostro utilizando CascadeClassifier

Como bien vimos en la presentación, uno de los primeros pasos en el reconocimiento facial es la detección de la cara. Para detectar la cara, utilizaremos un clasificador llamado CascadeClassifier, que nos pedirá como argumento un modelo de detección ya entrenado. El enlace de descarga para descargar este modelo se encuentra en la siguiente casilla.

Para que el clasificador pueda detectar la cara, necesita trabajar con imágenes en escala de grises, de igual manera se debe realizar como primer paso recorte de la ubicación del rostro dentrro de la imagen. En este fragmento de código se realiza la conversión de la imagen a escala de grises y se utilizan el cv2.CascadeClassifier mediante un modelo de detección de frontal de rostros, para realizar el recorte de la cara en la imagen. Este método devuelve el (x, y, width, height) de la zona en la cual se encuentra el rostro, para que pueda ser extraida utilizando OpenCV.

Enlace de Descarga del Modelo de detección: 
haarCascade: https://drive.google.com/file/d/1TtVphkjlkpCQRrK7Tm1dzShfdCsScWhH/view?usp=sharing
lbpCascade: https://drive.google.com/file/d/1fAg-1sSn7wpqQJkXObzkhRdW9ZrUBy1e/view?usp=sharing

In [ ]:
#función para detectar la cara usando OpenCV
def detect_face(img):
    #convierte la imagen de prueba en una imagen en escala de grises, ya que el detector de rostros opencv trabaja con 
    #imágenes en escala de grises
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    #cargamos el clasificador Cascade, utilizando OpenCV. Usaremos un archivo xml que contiene el algoritmo de
    #detección del rostro ya entrenado para saber qué características extraer de la imagen para encontrar el rostro
    #Usaremos el lbpcascade, que es un modelo más eficiente que sus equivalentes.
    face_cascade = cv2.CascadeClassifier('opencv/lbpcascade_frontalface.xml')

    #detectemos imágenes multiescala (algunas imágenes pueden estar más cerca de la cámara que otras)
    #El resultado es la lista de caras encontradas
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5);
    
    #si no se detectan caras, la condición hará que el programa retorne la imagen original
    if (len(faces) == 0):
        return None, None
    
    #bajo el supuesto de que solo habrá una cara, se extraerá el área de la cara
    (x, y, w, h) = faces[0]
    
    gray = cv2.resize(gray[y:y+w, x:x+h],(224,224)) #se cambiará el tamaño de la cara recortada en un tamaño 224x224
    
    #devolverá solo la parte de la cara de la imagen
    return gray, faces[0]

### Localización de la cara y recorte 

Para poder entrenar el modelo, necesitamos obtener las caras que vamos a detectar y recortar. Cuando ya estén recortadas, lo siguiente que se hará es colocar las etiquetas a cada rostro detectado. En resumidas cuentas, esa es el funcionamiento del siguiente fragmento de código. La función recorrerá el directorio que le enviemos como parámetro y obtendrá cada imagen a la que se le detectará la cara para posteriormente etiquetarla. Básicamente, preparará los datos para su posterior entrenamiento.

Enlace del dataset: https://drive.google.com/drive/folders/1C0-UxcGntsAxVgYbmIvzv6HoOf06M1TW?usp=sharing

In [ ]:
#esta función leerá las imágenes de entrenamiento de todas las personas, detectará la cara de cada imagen
#y retornará dos listas exactamente del mismo tamaño, una lista de caras y otra lista de etiquetas para cada cara
def prepare_training_data(data_folder_path):
    
    #------PASO-1--------
    #obtener los directorios (un directorio para cada sujeto (clase)) en la carpeta de entrenamiento
    dirs = os.listdir(data_folder_path)
    
    #lista para almacenar todas las caras del sujeto
    faces = []
    #lista para contener todas las etiquetas de la cara del sujeto
    labels = []
    
    #mediante un ciclo repetitivo, buscaremos las carpetas dentro de nuestra carpeta raíz (training-data)
    for dir_name in dirs:
        
        #las carpetas que contienen las caras de nuestros sujetos inician con la letra 's' por lo que
        #debemos ignorar las carpetas que se encuentren dentro del directorio raíz si no inicia con la letra
        #'s'
        if not dir_name.startswith("s"):
            continue;
            
        #------PASO-2--------
        #extraer el número de etiqueta del sujeto de dir_name
        #formato del directorio name = slabel
        #, por lo que eliminar la letra 's' de dir_name nos dará una etiqueta
        label = int(dir_name.replace("s", ""))
        
        #construye la ruta del directorio que contiene imágenes para el tema actual
        subject_dir_path = data_folder_path + "/" + dir_name
        
        #obtenemos los nombres de las imágenes que están dentro del directorio de los sujetos dados
        subject_images_names = os.listdir(subject_dir_path)
        
        #------PASO-3--------
        #iremos a través de cada nombre de imagen, leeremos la imagen,
        #detectaremos el rostro y agregaremos rostro a la lista de rostros
        for image_name in subject_images_names:
            
            #ignoramos los archivos del sistema como .DS_Store
            if image_name.startswith("."):
                continue;
            
            #creamos la ruta de la imagen
            #ejemplo de una ruta de imagen = training-data/s1/1.jpg
            image_path = subject_dir_path + "/" + image_name

            #leemos la imagen obtenida de la ruta
            image = cv2.imread(image_path)
            
            #desplegamos una ventana para que contenga las imágenes que se están obteniendo de la ruta
            #una por una
            cv2.imshow("Training on image...", image)
            cv2.waitKey(100)
            
            #detectamos la cara de cada imagen desplegada
            face, rect = detect_face(image)
            
            #------Paso-4--------
            #En este ejemplo, ignoraremos las caras no detectadas
            if face is not None:
                #añadimos la cara detectada a la lista de caras
                faces.append(face)
                #añadimos la etiqueta para esta cara
                labels.append(label)
            
    cv2.destroyAllWindows() #Cerramos la ventana desplegada para mostrar las imágenes dentro de la ruta de entrenamiento
    cv2.waitKey(1)
    cv2.destroyAllWindows()
    
    return faces, labels

Enviaremos a la función el parámetro, que en este caso es el nombre de la carpeta con el dataset. Puede variar en muchos casos esta ruta, dependiendo de donde tenemos almacenado el dataset.

In [ ]:
#Primeramente, prepararemos nuestros datos de entrenamiento
#Los datos estarán en dos listas del mismo tamaño
#una lista contrendrá todas las caras detectadas
#y la otra lista contendrá las respectivas etiquetas para cada cara
print("Preparing data...")
faces, labels = prepare_training_data("training-data")
print("Data prepared")

#imprimimos el total de caras y etiquetas encontradas
print("Total faces: ", len(faces))
print("Total labels: ", len(labels))

### Entrenamiento del Modelo 

En este paso procedemos a instanciar el reconocedor y posteriormente se realiza el entrenamiento del mismo utilizando el método `train(faces-vector, labels-vector)` el cual recibe la lista de caras y etiquetas del conjunto de entrenamiento.

In [ ]:
#Para usar el EigenFaceRecognizer  
#face_recognizer = cv2.face.EigenFaceRecognizer_create()

#Para utilizar el FisherFaceRecognizer 
face_recognizer = cv2.face.FisherFaceRecognizer_create()

#Para usar el LBPHFaceRecognizer
#face_recognizer = cv2.face.LBPHFaceRecognizer_create()

#entrenamiento de nuestro modelo de reconocimiento facial utilizando las caras recortadas en la preparación del entrenamiento
face_recognizer.train(faces, np.array(labels))

Podemos también guardar nuestro modelo ya entrenado, para así maximizar y optimizar nuestro modelo a nivel de ejecución. Para esto usaremos el método save, que guardará en el formato que le indiquemos nuestro modelo entrenado. En este caso, guardaremos el modelo en formato yml. Podemos asignarle una carpeta o podemos almacenarlo en la misma ubicación de este mismo Notebook.

In [ ]:
face_recognizer.save('facialrecognitiontraining.yml')

El siguiente fragmento de código muestra cómo crear un rectángulo sobre la zona donde tanto el detector Cascade como el algoritmo de reconocimiento encontró la cara de la persona, y escribirán la etiqueta que ha definido el detector que corresponde con el rostro.

In [ ]:
#función que dibuja un rectángulo sobre la cara de la persona 
#de acuerdo con las coordenadas (x,y)
#y con el ancho y alto dados
def draw_rectangle(img, rect):
    (x, y, w, h) = rect
    cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
    
#función que escribe un texto sobre la imagen dada
#tomando las coordenadas (x, y). 
def draw_text(img, text, x, y):
    cv2.putText(img, text, (x, y), cv2.FONT_HERSHEY_PLAIN, 1.5, (0, 255, 0), 2)

### Predicción 

Para la predicción, utilizaremos una función que tomará como entrada la imagen cargada. La función hará una copia de la imagen original para no afectarla y la enviará a la función `detect_face`, donde esta función tomará la imagen copiada, la transformará a escala de grises y obtendrá la zona donde se encuentra el rostro (mediante coordenadas x y y) y las comparará con las clases ya entrenadas en el dataset.

In [ ]:
#esta función reconoce a la persona en la imagen suministrada
#y dibuja un rectángulo alrededor de la cara detectada con el nombre de la 
#persona 
def predict(test_img):
    #hace una copia de la imagen, ya que no queremos cambiar la imagen original
    img = test_img.copy()
    #detectamos la cara en la imagen
    face, rect = detect_face(img)

    #predecimos a la persona de la imagen usando nuestro reconocedor de rostros
    label= face_recognizer.predict(face)
    #print(label[0])
    #obtenemos el nombre de la etiqueta respectiva devuelta por el reconocedor de rostros
    label_text = subjects[label[0]]
    
    #dibujamos el rectangulo alrededor de la cara detectada
    draw_rectangle(img, rect)
    #Insertamos el nombre de la persona reconocida
    draw_text(img, label_text, rect[0], rect[1]-5)
    
    return img #retornamos la imagen con el reconocimiento del rostro

Posteriormente, enviaremos la o las imágenes que queremos probar para ver si nuestro modelo funciona correctamente

Enlace de las imágenes para las pruebas: https://drive.google.com/drive/folders/1vEZbarLQzZgnHFgdDgsvPkiZwgKPsTkM?usp=sharing

In [ ]:
print("Predicting images...")

#Cargamos las imágenes de pruebas
test_img1 = cv2.imread("test-data/test8.jpg")
test_img2 = cv2.imread("test-data/test17.jpg")

#hacemos una predicción
predicted_img1 = predict(test_img1)
predicted_img2 = predict(test_img2)
print("Prediction complete")

#Desplegamos los resultados obtenidos para las imágenes de pruebas suministradas
cv2.imshow('Sujeto1', predicted_img1)
cv2.imshow('Sujeto2', predicted_img2)
cv2.waitKey(0)
cv2.destroyAllWindows() #cerramos las ventanas desplegadas

## Ejemplo 2 

En el ejemplo anterior, pudimos observar cómo funciona el reconocimiento facial a nivel de codificación, pero usando imágenes cargada desde la computadora. En este ejemplo, realizaremos una detección y reconocimiento facial en tiempo real, utilizando la cámara web de nuestra computadora. Usaremos un dataset con la cara del presentador y de otras personas, para que se muestre el reconocimiento de varias personas en tiempo real. 

### Importación de Librerías y Etiquetado de Clases 

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import os

#etiquetado de cada sujeto a los que evaluaremos con el reconocimiento facial. El primer espacio de este vector lo dejamos
#siempre vacio
subjects = ["", "Ruben Blades", "Miguel Vasquez", "Román Torres", "Erika Ender"]

### Detección de la cara 

In [ ]:
#función para detectar la cara usando OpenCV
def detect_face(img):
    #convierte la imagen de prueba en una imagen en escala de grises, ya que el detector de rostros opencv trabaja con 
    #imágenes en escala de grises
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    #cargamos el clasificador Cascade, utilizando OpenCV. Usaremos un archivo xml que contiene el algoritmo de
    #detección del rostro ya entrenado para saber qué características extraer de la imagen para encontrar el rostro
    #Usaremos el lbpcascade, que es un modelo más eficiente que sus equivalentes.
    face_cascade = cv2.CascadeClassifier('opencv/lbpcascade_frontalface.xml')

    #detectemos imágenes multiescala (algunas imágenes pueden estar más cerca de la cámara que otras)
    #El resultado es la lista de caras encontradas
    
    #PARÁMETROS A TOMAR EN CUENTA
    #scaleFactor = Este valor debe ser mayor a 1, pero entre más cerca esta del 1
    #Mayor tiempo toma para entrenar, pero así reconoce mejor a la persona
    #minNeighbors = el valor de 3 esta bien, pero entre mayor es, menos reconocerá.
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.03, minNeighbors=3);
    
    #si no se detectan caras, la condición hará que el programa retorne la imagen original
    if (len(faces) == 0):
        return None, None
    
    #bajo el supuesto de que solo habrá una cara, se extraerá el área de la cara
    (x, y, w, h) = faces[0]
    
    gray = cv2.resize(gray[y:y+w, x:x+h],(224,224)) #se cambiará el tamaño de la cara recortada en un tamaño 224x224
    
    #retornaremos solamente la parte de la cara de la imagen
    return gray, faces[0]

### Extracción de caras del dataset y etiquetado

In [ ]:
#esta función leerá las imágenes de entrenamiento de todas las personas, detectará la cara de cada imagen
#y retornará dos listas exactamente del mismo tamaño, una lista de caras y otra lista de etiquetas para cada cara
def prepare_training_data(data_folder_path):
    
    #------PASO-1--------
    #obtener los directorios (un directorio para cada sujeto (clase)) en la carpeta de entrenamiento
    dirs = os.listdir(data_folder_path)
    
    #lista para almacenar todas las caras del sujeto
    faces = []
    #lista para contener todas las etiquetas de la cara del sujeto
    labels = []
    
    #mediante un ciclo repetitivo, buscaremos las carpetas dentro de nuestra carpeta raíz (training-data-project)
    for dir_name in dirs:
        
        #las carpetas que contienen las caras de nuestros sujetos inician con la letra 's' por lo que
        #debemos ignorar las carpetas que se encuentren dentro del directorio raíz si no inicia con la letra
        #'s'
        if not dir_name.startswith("s"):
            continue;
            
        #------PASO-2--------
        #extraer el número de etiqueta del sujeto de dir_name
        #formato del directorio name = slabel
        #, por lo que eliminar la letra 's' de dir_name nos dará una etiqueta
        label = int(dir_name.replace("s", ""))
        
        #construye la ruta del directorio que contiene imágenes para el tema actual
        subject_dir_path = data_folder_path + "/" + dir_name
        
        #obtenemos los nombres de las imágenes que están dentro del directorio de los sujetos dados
        subject_images_names = os.listdir(subject_dir_path)
        
        #------PASO-3--------
        #iremos a través de cada nombre de imagen, leeremos la imagen,
        #detectaremos el rostro y agregaremos rostro a la lista de rostros
        for image_name in subject_images_names:
            
            #ignoramos los archivos del sistema como .DS_Store
            if image_name.startswith("."):
                continue;
            
            ##creamos la ruta de la imagen
            #ejemplo de una ruta de imagen = training-data/s1/1.jpg
            image_path = subject_dir_path + "/" + image_name

            #leemos la imagen obtenida de la ruta
            image = cv2.imread(image_path)
            
            #desplegamos una ventana para que contenga las imágenes que se están obteniendo de la ruta
            #una por una 
            cv2.imshow("Training on image...", image)
            cv2.waitKey(100)
            
            #detectamos el rostro
            face, rect = detect_face(image)
            
            #------Paso-4--------
            #En este ejemplo, ignoraremos las caras no detectadas
            if face is not None:
                #añadimos la cara detectada a la lista de caras
                faces.append(face)
                ##añadimos la etiqueta de la cara detectada a la lista de etiquetas
                labels.append(label)
            
    cv2.destroyAllWindows()
    cv2.waitKey(1)
    cv2.destroyAllWindows()
    
    return faces, labels

Enlace del dataset: https://drive.google.com/drive/folders/1HvldSIyEAhwfZ_HrDaeFM4PnalDGeIe3?usp=sharing

In [ ]:
#Primeramente, prepararemos nuestros datos de entrenamiento
#Los datos estarán en dos listas del mismo tamaño
#una lista contrendrá todas las caras detectadas
#y la otra lista contendrá las respectivas etiquetas para cada cara
print("Preparing data...")
faces, labels = prepare_training_data("training-data-project")
print("Data prepared")

#imprimimos el total de caras y etiquetas encontradas
print("Total faces: ", len(faces))
print("Total labels: ", len(labels))

### Entrenamiento del Modelo

In [ ]:
#Para usar el EigenFaceRecognizer  
#face_recognizer = cv2.face.EigenFaceRecognizer_create()

#Para utilizar el FisherFaceRecognizer 
face_recognizer = cv2.face.FisherFaceRecognizer_create()

#Para usar el LBPHFaceRecognizer
#face_recognizer = cv2.face.LBPHFaceRecognizer_create()

#entrenamiento de nuestro modelo de reconocimiento facial utilizando las caras recortadas en la preparación del entrenamiento
face_recognizer.train(faces, np.array(labels))

### Captura de imagen usando la cámara web

A diferencia del ejemplo pasado, esta vez tomaremos la imagen en tiempo real. Para ello, usaremos un método de la librería cv2, que permite abrir una conexión con la cámara web o una cámara externa que se encuentre conectada a la computadora. El método se llama `VideoCapture` y su único parámetro es el número de cámara que vamos a utilizar. Si solo contamos con una cámara incorporada en nuestra computadora, su numeración es o 0 o 1. Para almacenar lo que se está viendo en tiempo real, usaremos el método `read()`.

In [ ]:
# creación del objeto VideoCapture para poder hacer uso de la cámara:
capture = cv2.VideoCapture(0)

face_cascade = cv2.CascadeClassifier('opencv/lbpcascade_frontalface.xml') #Lectura del archivo xml con el detector Cascade

while True:
    # Captura frame por frame desde la salida de video capturado por el objeto 'capture':
    ret, frame = capture.read()

    # Conversión de lo capturado en el frame a escala de grises para poder realizar la detección:
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    aux_frame = gray_frame.copy()
    
    faces = face_cascade.detectMultiScale(gray_frame,1.1,4)  #Detección del rostro de la persona
    
    #Colocar un recuadro azul sobre la cara detectada
    for(x,y,w,h) in faces:
        rostro = aux_frame[y:y+h,x:x+w]
        rostro = cv2.resize(rostro,(224,224))
        result = face_recognizer.predict(rostro)
        label_text = subjects[result[0]]
        #cv2.putText(frame, label_text, (x, y), cv2.FONT_HERSHEY_PLAIN, 1.5, (0, 255, 0), 2)
        
        if result[1] < 6000:
            #print('Valor: ',result[1])
            cv2.putText(frame, label_text, (x, y), cv2.FONT_HERSHEY_PLAIN, 1.5, (0, 255, 0), 2)
            cv2.rectangle(frame,(x,y),(x+w, y+h),(255,0,0),2)
        else:
            #print('Valor: ',result[1])
            cv2.putText(frame, 'Desconocido', (x, y), cv2.FONT_HERSHEY_PLAIN, 1.5, (0, 255, 0), 2)
            cv2.rectangle(frame,(x,y),(x+w, y+h),(0,0,255),2)
        
        #Colocar un texto sobre la imágen de salida (Se colocará en la esquina superior izquierda)
    cv2.putText(img=frame,
                text="Reconocimiento facial",
                org=(10, 40),
                fontFace=2,
                fontScale=1,
                color=(255, 255, 255),
                thickness=3)
    
    # Se muestra la imágen de salida en tiempo real
    cv2.imshow('frame', frame)
    
    #Cerrar la imagen de salida a partir del clic sobre la tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
# Cerrar las ventanas y borrar el contenido del objeto capture:
capture.release()
cv2.destroyAllWindows()

Para deshabilitar la cámara y cerrar el detector, debemos presionar la tecla q. Si cerramos en la cruz, la cámara seguirá activa, ya que la conexión solo se cerrará cuando cliquemos la letra q. 